<h1><center>Row data</center></h1>

In [19]:
import pandas as pd
import sqlite3
import datetime
import numpy as np
import os
import glob
import re
from os.path import join, getsize
from datetime import datetime
pd.set_option('precision', 2)

#連資料庫
sqldb = "BiobankData.db"
conn = sqlite3.connect(sqldb)
cursor = conn.cursor()

#檢查目標母資料夾
forward_path = './tmp_json_central/'
if not os.path.isdir(forward_path):
    os.mkdir(forward_path)

#疾病定義
group_disease_mapping = {
    
    '口腔癌': ["C00","C000","C001","C002","C003","C004","C005","C006","C008","C009","C01","C02","C020",
              "C021","C022","C023","C024","C028","C029","C03","C030","C031","C039","C04","C040","C041",
              "C048","C049","C05","C050","C051","C052","C058","C059","C06","C060","C061","C062","C068",
              "C0680","C0689","C069","C09","C090","C091","C098","C099","C10","C100","C101","C102","C103",
              "C104","C108","C109","C12","C13","C130","C131","C132","C138","C139","C14","C140","C142","C148"],

    '白血病': ["C42","C420","C421","C91","C910","C9100","C9101","C9102","C911","C9110","C9111","C9112",
             "C913","C9130","C9131","C9132","C914","C9140","C9141","C9142","C915","C9150","C9151","C9152",
             "C916","C9160","C9161","C9162","C919","C9190","C9191","C9192","C91A","C91A0","C91A1","C91A2",
             "C91Z","C91Z0","C91Z1","C91Z2","C92","C920","C9200","C9201","C9202","C921","C9210","C9211",
             "C9212","C922","C9220","C9221","C9222","C923","C9230","C9231","C9232","C924","C9240","C9241",
             "C9242","C925","C9250","C9251","C9252","C926","C9260","C9261","C9262","C929","C9290","C9291",
             "C9292","C92A","C92A0","C92A1","C92A2","C92Z","C92Z0","C92Z1","C92Z2","C93","C930","C9300",
             "C9301","C9302","C931","C9310","C9311","C9312","C933","C9330","C9331","C9332","C939","C9390",
             "C9391","C9392","C93Z","C93Z0","C93Z1","C93Z2","C94","C940","C9400","C9401","C9402","C942",
             "C9420","C9421","C9422","C943","C9430","C9431","C9432","C944","C9440","C9441","C9442","C946",
             "C948","C9480","C9481","C9482","C95","C950","C9500","C9501","C9502","C951","C9510","C9511",
             "C9512","C959","C9590","C9591","C9592"],

    '肝外膽管癌': ["C24","C240","C241","C248","C249"],

    '肝癌': ["C22","C220","C221","C222","C223","C224","C227","C228","C229"],

    '乳癌': ["C50","C500","C5001","C50011","C50012","C50019","C5002","C50021","C50022","C50029","C501",
             "C5011","C50111","C50112","C50119","C5012","C50121","C50122","C50129","C502","C5021","C50211",
             "C50212","C50219","C5022","C50221","C50222","C50229","C503","C5031","C50311","C50312","C50319",
             "C5032","C50321","C50322","C50329","C504","C5041","C50411","C50412","C50419","C5042","C50421",
             "C50422","C50429","C505","C5051","C50511","C50512","C50519","C5052","C50521","C50522","C50529",
             "C506","C5061","C50611","C50612","C50619","C5062","C50621","C50622","C50629","C508","C5081",
             "C50811","C50812","C50819","C5082","C50821","C50822","C50829","C509","C5091","C50911","C50912",
             "C50919","C5092","C50921","C50922","C50929"],

    '肺癌': ["C33","C34","C340","C3400","C3401","C3402","C341","C3410","C3411","C3412","C342","C343","C3430",
             "C3431","C3432","C348","C3480","C3481","C3482","C349","C3490","C3491","C3492"],

    '胃癌': ["C16","C160","C161","C162","C163","C164","C165","C166","C168","C169"],

    '食道癌': ["C15","C153","C154","C155","C158","C159"],

    '胰臟癌': ["C25","C250","C251","C252","C253","C254","C257","C258","C259"],

    '腸癌': ["C17","C170","C171","C172","C173","C178","C179","C18","C180","C181","C182","C183","C184",
             "C185","C186","C187","C188","C189","C19","C20","C21","C210","C211","C212","C218"],

    '膽囊癌': ["C23","for_tuple_fill"],

    '卵巢癌': ["C56","C561","C562","C563","C569"],

    '糖尿病': ["E08","E080","E0800","E0801","E081","E0810","E0811","E082","E0821","E0822","E0829","E083","E0831",
             "E08311","E08319","E0832","E08321","E08329","E0833","E08331","E08339","E0834","E08341","E08349",
             "E0835","E08351","E08359","E0836","E0839","E084","E0840","E0841","E0842","E0843","E0844","E0849",
             "E085","E0851","E0852","E0859","E086","E0861","E08610","E08618","E0862","E08620","E08621","E08622",
             "E08628","E0863","E08630","E08638","E0864","E08641","E08649","E0865","E0869","E088","E089","E09",
             "E090","E0900","E0901","E091","E0910","E0911","E092","E0921","E0922","E0929","E093","E0931","E09311",
             "E09319","E0932","E09321","E09329","E0933","E09331","E09339","E0934","E09341","E09349","E0935","E09351",
             "E09359","E0936","E0939","E094","E0940","E0941","E0942","E0943","E0944","E0949","E095","E0951","E0952",
             "E0959","E096","E0961","E09610","E09618","E0962","E09620","E09621","E09622","E09628","E0963","E09630",
             "E09638","E0964","E09641","E09649","E0965","E0969","E098","E099","E11","E110","E1100","E1101","E112",
             "E1121","E1122","E1129","E113","E1131","E11311","E11319","E1132","E11321","E11329","E1133","E11331",
             "E11339","E1134","E11341","E11349","E1135","E11351","E11359","E1136","E1139","E114","E1140","E1141",
             "E1142","E1143","E1144","E1149","E115","E1151","E1152","E1159","E116","E1161","E11610","E11618","E1162"
             ,"E11620","E11621","E11622","E11628","E1163","E11630","E11638","E1164","E11641","E11649","E1165","E1169",
             "E118","E119","E13","E130","E1300","E1301","E131","E1310","E1311","E132","E1321","E1322","E1329","E133",
             "E1331","E13311","E13319","E1332","E13321","E13329","E1333","E13331","E13339","E1334","E13341","E13349",
             "E1335","E13351","E13359","E1336","E1339","E134","E1340","E1341","E1342","E1343","E1344","E1349","E135",
             "E1351","E1352","E1359","E136","E1361","E13610","E13618","E1362","E13620","E13621","E13622","E13628","E1363",
             "E13630","E13638","E1364","E13641","E13649","E1365","E1369","E138","E139"],

    '慢性腎病': ["A1811","A5275","C64","C65","C689","D300","D3A093","D410","D411","D422","D593","E082","E092","E1021",
             "E1022","E1029","E1065","E112","E1165","E132","E260","E261","E268","E269","E7203","E723","E728","E740",
             "E744","E748","E7521","E7522","E7524","E753","E77","I12","I13","I701","I722","I7581","I773","I7773",
             "K767","M103","M1A10X0","M1A10X1","M1A1110","M1A1111","M1A1120","M1A1121","M1A1190","M1A1191","M1A1210",
             "M1A1211","M1A1220","M1A1221","M1A1290","M1A1291","M1A1310","M1A1311","M1A1320","M1A1321","M1A1390",
             "M1A1391","M1A1410","M1A1411","M1A1420","M1A1421","M1A1490","M1A1491","M1A1510","M1A1511","M1A1520",
             "M1A1521","M1A1590","M1A1591","M1A1610","M1A1611","M1A1620","M1A1621","M1A1690","M1A1691","M1A1710",
             "M1A1711","M1A1720","M1A1721","M1A1790","M1A1791","M1A18X0","M1A18X1","M1A19X0","M1A19X1","M3214",
             "M3215","M3504","N00","N008","N009","N01","N02","N03","N04","N05","N06","N07","N08","N131","N132",
             "N133","N135","N139","N14","N150","N158","N159","N16","N17","N18","N19","N200","N25","N261","N269",
             "N271","N279","N289","N29","O104","O121","O2683","Q60","Q6102","Q611","Q612","Q613","Q614","Q615",
             "Q618","Q620","Q621","Q622","Q6231","Q6232","Q6239","Q63","Q851","R944","T560X1A","T560X2A","T560X3A",
             "T560X4A","Z4822","Z524","Z940","Z99"],
    
    '阿茲海默症': ["G30","G300","G01","G08","G09"]

}

#計算Row data
for disease_group, codes in group_disease_mapping.items():
    
    #檢查/創立各病儲存夾
    save_path = forward_path + disease_group
    if not os.path.isdir(save_path):
        os.mkdir(save_path)
    
    #聯集三表間id
    CRLF = pd.read_sql("SELECT id, site FROM CRLF", conn)
    CRLF = CRLF[CRLF['site'].isin(codes)]
    CRLF_id_list = CRLF['id'].tolist()
    
    TOTFAE = pd.read_sql("SELECT d3, d19, d20, d21, d22, d23 FROM TOTFAE", conn)
    TOTFAE = TOTFAE[TOTFAE[['d19', 'd20', 'd21', 'd22', 'd23']].isin(codes).any(axis=1)]
    TOTFAE_id_list = TOTFAE['d3'].tolist()
    
    TOTFBE = pd.read_sql("SELECT d3, d25, d26, d27, d28, d29 FROM TOTFBE", conn)
    TOTFBE = TOTFBE[TOTFBE[['d25', 'd26', 'd27', 'd28', 'd29']].isin(codes).any(axis=1)]
    TOTFBE_id_list = TOTFBE['d3'].tolist()
    
    union_set = set(CRLF_id_list).union(TOTFAE_id_list, TOTFBE_id_list)
    union_list = list(union_set)
    
    #以id查詢/儲存資料
    CRLF_save_query = "SELECT * FROM CRLF WHERE id IN ({})".format(','.join('?' for _ in union_list))
    CRLF_save = pd.read_sql(CRLF_save_query, conn, params=union_list)
    CRLF_save = CRLF_save.to_json(save_path+"\\CRLF.json",orient='records',date_format = 'iso')
    
    TOTFAE_save_query = "SELECT * FROM TOTFAE WHERE d3 IN ({})".format(','.join('?' for _ in union_list))
    TOTFAE_save = pd.read_sql(TOTFAE_save_query, conn, params=union_list)
    TOTFAE_save = TOTFAE_save.to_json(save_path+"\\TOTFAE.json",orient='records',date_format = 'iso')
    
    TOTFBE_save_query = "SELECT * FROM TOTFBE WHERE d3 IN ({})".format(','.join('?' for _ in union_list))
    TOTFBE_save = pd.read_sql(TOTFBE_save_query, conn, params=union_list)
    TOTFBE_save = TOTFBE_save.to_json(save_path+"\\TOTFBE.json",orient='records',date_format = 'iso')
    
    TOTFAO1_d_save_query = "SELECT * FROM TOTFAO1 WHERE d3 IN ({}) AND LENGTH(p4) = 10".format(','.join('?' for _ in union_list))
    TOTFAO1_d_save = pd.read_sql(TOTFAO1_d_save_query, conn, params=union_list)
    TOTFAO1_d_save = TOTFAO1_d_save.to_json(save_path+"\\TOTFAO1_d.json",orient='records',date_format = 'iso')
    
    TOTFAO1_sc_save_query = "SELECT * FROM TOTFAO1 WHERE d3 IN ({}) AND LENGTH(p4) BETWEEN 6 AND 7".format(','.join('?' for _ in union_list))
    TOTFAO1_sc_save = pd.read_sql(TOTFAO1_sc_save_query, conn, params=union_list)
    TOTFAO1_sc_save = TOTFAO1_sc_save.to_json(save_path+"\\TOTFAO1_sc.json",orient='records',date_format = 'iso')
    
    TOTFAO1_o_save_query = "SELECT * FROM TOTFAO1 WHERE d3 IN ({}) AND LENGTH(p4) NOT IN (6, 7, 10)".format(','.join('?' for _ in union_list))
    TOTFAO1_o_save = pd.read_sql(TOTFAO1_o_save_query, conn, params=union_list)
    TOTFAO1_o_save = TOTFAO1_o_save.to_json(save_path+"\\TOTFAO1_o.json",orient='records',date_format = 'iso')
    
    TOTFAO2_save_query = "SELECT * FROM TOTFAO2 WHERE d3 IN ({})".format(','.join('?' for _ in union_list))
    TOTFAO2_save = pd.read_sql(TOTFAO2_save_query, conn, params=union_list)
    TOTFAO2_save = TOTFAO2_save.to_json(save_path+"\\TOTFAO2.json",orient='records',date_format = 'iso')
    
    TOTFBO1_d_save_query = "SELECT * FROM TOTFBO1 WHERE d3 IN ({}) AND LENGTH(p3) = 10".format(','.join('?' for _ in union_list))
    TOTFBO1_d_save = pd.read_sql(TOTFBO1_d_save_query, conn, params=union_list)
    TOTFBO1_d_save = TOTFBO1_d_save.to_json(save_path+"\\TOTFBO1_d.json",orient='records',date_format = 'iso')
    
    TOTFBO1_sc_save_query = "SELECT * FROM TOTFBO1 WHERE d3 IN ({}) AND LENGTH(p3) BETWEEN 6 AND 7".format(','.join('?' for _ in union_list))
    TOTFBO1_sc_save = pd.read_sql(TOTFBO1_sc_save_query, conn, params=union_list)
    TOTFBO1_sc_save = TOTFBO1_sc_save.to_json(save_path+"\\TOTFBO1_sc.json",orient='records',date_format = 'iso')
    
    TOTFBO1_o_save_query = "SELECT * FROM TOTFBO1 WHERE d3 IN ({}) AND LENGTH(p3) NOT IN (6, 7, 10)".format(','.join('?' for _ in union_list))
    TOTFBO1_o_save = pd.read_sql(TOTFBO1_o_save_query, conn, params=union_list)
    TOTFBO1_o_save = TOTFBO1_o_save.to_json(save_path+"\\TOTFBO1_o.json",orient='records',date_format = 'iso')
    
    TOTFBO2_save_query = "SELECT * FROM TOTFBO2 WHERE d3 IN ({})".format(','.join('?' for _ in union_list))
    TOTFBO2_save = pd.read_sql(TOTFBO2_save_query, conn, params=union_list)
    TOTFBO2_save = TOTFBO2_save.to_json(save_path+"\\TOTFBO2.json",orient='records',date_format = 'iso')
    
    LABM1_save_query = "SELECT * FROM LABM1 WHERE h9 IN ({})".format(','.join('?' for _ in union_list))
    LABM1_save = pd.read_sql(LABM1_save_query, conn, params=union_list)
    LABM1_save = LABM1_save.to_json(save_path+"\\LABM1.json",orient='records',date_format = 'iso')
    
    LABM2_save_query = "SELECT * FROM LABM2 WHERE h9 IN ({})".format(','.join('?' for _ in union_list))
    LABM2_save = pd.read_sql(LABM2_save_query, conn, params=union_list)
    LABM2_save = LABM2_save.to_json(save_path+"\\LABM2.json",orient='records',date_format = 'iso')
    
    CASE_save_query = "SELECT * FROM [CASE] WHERE id IN ({})".format(','.join('?' for _ in union_list))
    CASE_save = pd.read_sql(CASE_save_query, conn, params=union_list)
    CASE_save = CASE_save.to_json(save_path+"\\CASE.json",orient='records',date_format = 'iso')
    
    DEATH_save_query = "SELECT * FROM DEATH WHERE id IN ({})".format(','.join('?' for _ in union_list))
    DEATH_save = pd.read_sql(DEATH_save_query, conn, params=union_list)
    DEATH_save = DEATH_save.to_json(save_path+"\\DEATH.json",orient='records',date_format = 'iso')

<h1><center>孟</center></h1>

In [35]:
#取得當前全部疾病夾路徑
directory_path = r'.\tmp_json_central'
folder_paths = [] 
for root, dirs, files in os.walk(directory_path):
    for dir_name in dirs:
        folder_path = os.path.join(root, dir_name)
        folder_paths.append(folder_path)

#主def
def process_table(disease_name, table_name, json_path, id_column, date_column):
    try:
        df = pd.read_json(json_path, orient='records')

        # 計算每個 'id' 的資料筆數
        id_counts = df[id_column].value_counts()
        
        # 計算每個 'id' 分男女的資料筆數
        if 'gender' in df.columns:
            gender_1_count = len(df[df['gender'] == 1][id_column].unique())
            gender_2_count = len(df[df['gender'] == 2][id_column].unique())
        else:
            gender_1_count = "-"
            gender_2_count = "-"

        # 計算統計數據
        result_df = pd.DataFrame({
            'disease_name': [disease_name],
            'table_name': [table_name],
            'N': [len(id_counts)],
            'N_male': [gender_1_count],
            'N_female': [gender_2_count],
            'Mean': [id_counts.mean()],
            'Min': [id_counts.min()],
            'Q1': [id_counts.quantile(0.25)],
            'Q2': [id_counts.quantile(0.5)],
            'Q3': [id_counts.quantile(0.75)],
            'Max': [id_counts.max()],
            'start_date': [df[date_column].min()],
            'end_date': [df[date_column].max()]
        })

    except:
        result_df = pd.DataFrame({
            'disease_name': [disease_name],
            'table_name': [table_name],
            'N': ["-"],
            'N_male': ["-"],
            'N_female': ["-"],
            'Mean': ["-"],
            'Min': ["-"],
            'Q1': ["-"],
            'Q2': ["-"],
            'Q3': ["-"],
            'Max': ["-"],
            'start_date': ["-"],
            'end_date': ["-"]
        })

    return result_df

        
## 各病開始呼叫主def計算
# 創建表格模板
column_names = ['disease_name', 'table_name', 'N', 'N_male', 'N_female', 'Mean', 'Min', 'Q1', 'Q2', 'Q3', 'Max', 'start_date', 'end_date']
final_summary_df = pd.DataFrame(columns=column_names)

#取code與date的對應欄位
tables = {
    'CRLF': ['CRLF.json', 'id', 'didiag'],
    'TOTFAE': ['TOTFAE.json', 'd3', 'd9'],
    'TOTFBE': ['TOTFBE.json', 'd3', 'd10'],
    'TOTFAO1_d':['TOTFAO1_d.json','d3','d9'],
    'TOTFAO1_sc':['TOTFAO1_sc.json','d3','d9'],
    'TOTFAO1_o':['TOTFAO1_o.json','d3','d9'],
    'TOTFAO2':['TOTFAO2.json','d3','d9'],
    'TOTFBO1_d':['TOTFBO1_d.json','d3','d9'],
    'TOTFBO1_sc':['TOTFBO1_sc.json','d3','d9'],
    'TOTFBO1_o':['TOTFBO1_o.json','d3','d9'],
    'TOTFBO2':['TOTFBO2.json','d3','d9'],
    'LABM1':['LABM1.json','h9','h13'],
    'LABM2':['LABM2.json','h9','h13'],
    'CASE':['CASE.json','id','m3'],
    'DEATH':['DEATH.json','id','d4'],
}

##疊/儲存表格
for path in folder_paths:
    
    #從資料夾名稱取出disease_name
    disease_name = os.path.basename(path)
    
    for table_name, (json_file, id_column, date_column) in tables.items():
        json_path = os.path.join(path, json_file)
        summary_df = process_table(disease_name, table_name, json_path, id_column, date_column)
        final_summary_df = pd.concat([final_summary_df, summary_df], ignore_index=True)

final_summary_df = final_summary_df.round(2)
final_summary_df_save = final_summary_df.to_json("./tmp_json_central/final_summary_df.json",orient='records',date_format = 'iso')
final_summary_df

,disease_name,table_name,N,N_male,N_female,Mean,Min,Q1,Q2,Q3,Max,start_date,end_date
0,乳癌,CRLF,80,3,77,1.04,1,1.0,1.0,1.0,3,20190107,20201230
1,乳癌,TOTFAE,80,3,77,36.5,5,20.0,29.0,50.0,135,20191022,20211231
2,乳癌,TOTFBE,80,3,77,5.11,1,1.0,2.0,8.0,26,20200103,20211230
3,乳癌,TOTFAO1_d,79,-,-,41.68,1,10.0,22.0,57.0,175,20191121,20211231
4,乳癌,TOTFAO1_sc,80,-,-,157.51,21,56.0,124.0,226.75,541,20191022,20211231
...,...,...,...,...,...,...,...,...,...,...,...,...,...
220,食道癌,TOTFBO2,-,-,-,-,-,-,-,-,-,-,-
221,食道癌,LABM1,1,0,1,585.0,585,585.0,585.0,585.0,585,,20210927
222,食道癌,LABM2,-,-,-,-,-,-,-,-,-,-,-
223,食道癌,CASE,-,-,-,-,-,-,-,-,-,-,-


<h1><center>修</center></h1>

In [32]:
import pandas as pd
import sqlite3
import datetime
import numpy as np
import os
import glob
import re
from os.path import join, getsize
from datetime import datetime
import pandas as pd
from pandasql import sqldf
from functools import partial

##找出母資料夾底下所有AO1、BO1
def search_files(directory, prefix_list):
    
    matching_files_AE = []
    matching_files_BE = []
    
    for root, dirs, files in os.walk(directory):
        for file in files:
            for prefix in prefix_list:
                if file.startswith(prefix):
                    if prefix == 'TOTFAO1':
                        matching_files_AE.append(os.path.join(root, file))
                    elif prefix == 'TOTFBO1':
                        matching_files_BE.append(os.path.join(root, file))
    
    return matching_files_AE, matching_files_BE

# 指定母資料夾路徑
directory = "./tmp_json_central"

# 指定檔案名稱的前綴
prefix_list = ['TOTFAO1', 'TOTFBO1']

# 搜尋符合條件的檔案並排除二跑既存_freq檔
matching_files_AE, matching_files_BE = search_files(directory, prefix_list)
matching_files_AE = [f for f in matching_files_AE if not re.search(r'_freq\.json$', f)]
matching_files_BE = [f for f in matching_files_BE if not re.search(r'_freq\.json$', f)]

##主def_AE
def sql_query_AE(json_path):
    
    df = pd.read_json(json_path, orient ='records')
    
    #宣告 pysqldf 函數通道 且 轉為local變數配合區域變數df
    pysqldf = partial(sqldf, env=locals())
        
    # SQL查詢語法
    query = """
        WITH patient_usage AS (
            SELECT 
                p4, 
                d3, 
                COUNT(*) as usage_count
            FROM 
                df
            GROUP BY 
                p4, d3
        ),
        total_usage AS (
            SELECT 
                p4,
                COUNT(*) as total_count
            FROM 
                df
            GROUP BY 
                p4
        ),
        quartile_stats AS (
            SELECT
                p4,
                AVG(usage_count) FILTER (WHERE quartile = 1) AS Q1,
                AVG(usage_count) FILTER (WHERE quartile = 2) AS Q2,
                AVG(usage_count) FILTER (WHERE quartile = 3) AS Q3,
                AVG(usage_count) FILTER (WHERE quartile = 4) AS Q4
            FROM (
                SELECT
                    p4,
                    usage_count,
                    NTILE(4) OVER (PARTITION BY p4 ORDER BY usage_count) AS quartile
                FROM 
                    patient_usage
            )
            GROUP BY p4
        ),
        order_stats AS (
            SELECT 
                pu.p4, 
                COUNT(*) as unique_users, 
                AVG(pu.usage_count) as avg_usage_per_user,
                MAX(pu.usage_count) as max_usage_per_user, 
                MIN(pu.usage_count) as min_usage_per_user,
                tu.total_count
            FROM 
                patient_usage pu
            JOIN 
                total_usage tu ON pu.p4 = tu.p4
            GROUP BY 
                pu.p4, tu.total_count
        ),
        time_stats AS (
            SELECT 
                p4, 
                MIN(t3) as first_usage_date, 
                MAX(t3) as last_usage_date
            FROM 
                df
            GROUP BY 
                p4
        )
        SELECT 
            o.p4, 
            o.unique_users, 
            o.avg_usage_per_user, 
            o.max_usage_per_user, 
            o.min_usage_per_user,
            o.total_count,
            q.Q1,
            q.Q2,
            q.Q3,
            q.Q4,
            t.first_usage_date, 
            t.last_usage_date
        FROM 
            order_stats o
        JOIN 
            quartile_stats q ON o.p4 = q.p4
        JOIN 
            time_stats t ON o.p4 = t.p4;
    """
    # 查詢
    try:
        result = pysqldf(query)
        result = result.round(2)

        # 另存新檔 base_name:路徑串 ; extension:副檔名
        base_name, extension = os.path.splitext(json_path)
        new_path = base_name + '_freq' + extension
        df_save = result.to_json(new_path,orient='records',date_format = 'iso')

        return('Success: ',new_path)
    
    except:
        result = pd.DataFrame()
        
        # 另存新檔 base_name:路徑串 ; extension:副檔名
        base_name, extension = os.path.splitext(json_path)
        new_path = base_name + '_freq' + extension
        df_save = result.to_json(new_path,orient='records',date_format = 'iso')

        return('Fail: ',new_path)
        
##主def_BE
def sql_query_BE(json_path):
    
    df = pd.read_json(json_path, orient ='records')
    
    #宣告 pysqldf 函數通道 且 轉為local變數配合區域變數df
    pysqldf = partial(sqldf, env=locals())
    
    # SQL查詢語法
    query = """
        WITH patient_usage AS (
            SELECT 
                p3, 
                d3, 
                COUNT(*) as usage_count
            FROM 
                df
            GROUP BY 
                p3, d3
        ),
        total_usage AS (
            SELECT 
                p3,
                COUNT(*) as total_count
            FROM 
                df
            GROUP BY 
                p3
        ),
        quartile_stats AS (
            SELECT
                p3,
                AVG(usage_count) FILTER (WHERE quartile = 1) AS Q1,
                AVG(usage_count) FILTER (WHERE quartile = 2) AS Q2,
                AVG(usage_count) FILTER (WHERE quartile = 3) AS Q3,
                AVG(usage_count) FILTER (WHERE quartile = 4) AS Q4
            FROM (
                SELECT
                    p3,
                    usage_count,
                    NTILE(4) OVER (PARTITION BY p3 ORDER BY usage_count) AS quartile
                FROM 
                    patient_usage
            )
            GROUP BY p3
        ),
        order_stats AS (
            SELECT 
                pu.p3, 
                COUNT(*) as unique_users, 
                AVG(pu.usage_count) as avg_usage_per_user,
                MAX(pu.usage_count) as max_usage_per_user, 
                MIN(pu.usage_count) as min_usage_per_user,
                tu.total_count
            FROM 
                patient_usage pu
            JOIN 
                total_usage tu ON pu.p3 = tu.p3
            GROUP BY 
                pu.p3, tu.total_count
        ),
        time_stats AS (
            SELECT 
                p3, 
                MIN(t3) as first_usage_date, 
                MAX(t3) as last_usage_date
            FROM 
                df
            GROUP BY 
                p3
        )
        SELECT 
            o.p3, 
            o.unique_users, 
            o.avg_usage_per_user, 
            o.max_usage_per_user, 
            o.min_usage_per_user,
            o.total_count,
            q.Q1,
            q.Q2,
            q.Q3,
            q.Q4,
            t.first_usage_date, 
            t.last_usage_date
        FROM 
            order_stats o
        JOIN 
            quartile_stats q ON o.p3 = q.p3
        JOIN 
            time_stats t ON o.p3 = t.p3;
    """ 
    # 查詢
    try:     
        result = pysqldf(query)
        result = result.round(2)

        # 另存新檔 base_name:路徑串 ; extension:副檔名
        base_name, extension = os.path.splitext(json_path)
        new_path = base_name + '_freq' + extension
        df_save = result.to_json(new_path,orient='records',date_format = 'iso')

        return(new_path)
    
    except:
        result = pd.DataFrame()

        # 另存新檔 base_name:路徑串 ; extension:副檔名
        base_name, extension = os.path.splitext(json_path)
        new_path = base_name + '_freq' + extension
        df_save = result.to_json(new_path,orient='records',date_format = 'iso')

        return('Fail: ',new_path)
        
# AE迴圈
for json_path in matching_files_AE:
    sql_query_AE(json_path)

# BE迴圈
for json_path in matching_files_BE:
    sql_query_BE(json_path)

<h1><center>py</center></h1>

In [2]:
import pandas as pd
import sqlite3
import datetime
import numpy as np
import os
import glob
import re
from os.path import join, getsize
from pandasql import sqldf
from functools import partial
import shutil
pd.set_option('precision', 2)

################################
#delect old zip
################################
oldzip_path = '.\\tmp_json_central_download\\central_result.zip'

# 如果存在删除文件
if os.path.exists(oldzip_path):
    os.remove(oldzip_path)

################################
#Create row data
################################

#連資料庫
sqldb = "BiobankData.db"
conn = sqlite3.connect(sqldb)
cursor = conn.cursor()

#檢查目標母資料夾與結果資料夾
forward_path = './tmp_json_central/'
if not os.path.isdir(forward_path):
    os.mkdir(forward_path)

download_path = './tmp_json_central_download/'
if not os.path.isdir(download_path):
    os.mkdir(download_path)

#疾病定義
group_disease_mapping = {
    
    '口腔癌': ["C00","C000","C001","C002","C003","C004","C005","C006","C008","C009","C01","C02","C020",
              "C021","C022","C023","C024","C028","C029","C03","C030","C031","C039","C04","C040","C041",
              "C048","C049","C05","C050","C051","C052","C058","C059","C06","C060","C061","C062","C068",
              "C0680","C0689","C069","C09","C090","C091","C098","C099","C10","C100","C101","C102","C103",
              "C104","C108","C109","C12","C13","C130","C131","C132","C138","C139","C14","C140","C142","C148"],

    '白血病': ["C42","C420","C421","C91","C910","C9100","C9101","C9102","C911","C9110","C9111","C9112",
             "C913","C9130","C9131","C9132","C914","C9140","C9141","C9142","C915","C9150","C9151","C9152",
             "C916","C9160","C9161","C9162","C919","C9190","C9191","C9192","C91A","C91A0","C91A1","C91A2",
             "C91Z","C91Z0","C91Z1","C91Z2","C92","C920","C9200","C9201","C9202","C921","C9210","C9211",
             "C9212","C922","C9220","C9221","C9222","C923","C9230","C9231","C9232","C924","C9240","C9241",
             "C9242","C925","C9250","C9251","C9252","C926","C9260","C9261","C9262","C929","C9290","C9291",
             "C9292","C92A","C92A0","C92A1","C92A2","C92Z","C92Z0","C92Z1","C92Z2","C93","C930","C9300",
             "C9301","C9302","C931","C9310","C9311","C9312","C933","C9330","C9331","C9332","C939","C9390",
             "C9391","C9392","C93Z","C93Z0","C93Z1","C93Z2","C94","C940","C9400","C9401","C9402","C942",
             "C9420","C9421","C9422","C943","C9430","C9431","C9432","C944","C9440","C9441","C9442","C946",
             "C948","C9480","C9481","C9482","C95","C950","C9500","C9501","C9502","C951","C9510","C9511",
             "C9512","C959","C9590","C9591","C9592"],

    '肝外膽管癌': ["C24","C240","C241","C248","C249"],

    '肝癌': ["C22","C220","C221","C222","C223","C224","C227","C228","C229"],

    '乳癌': ["C50","C500","C5001","C50011","C50012","C50019","C5002","C50021","C50022","C50029","C501",
             "C5011","C50111","C50112","C50119","C5012","C50121","C50122","C50129","C502","C5021","C50211",
             "C50212","C50219","C5022","C50221","C50222","C50229","C503","C5031","C50311","C50312","C50319",
             "C5032","C50321","C50322","C50329","C504","C5041","C50411","C50412","C50419","C5042","C50421",
             "C50422","C50429","C505","C5051","C50511","C50512","C50519","C5052","C50521","C50522","C50529",
             "C506","C5061","C50611","C50612","C50619","C5062","C50621","C50622","C50629","C508","C5081",
             "C50811","C50812","C50819","C5082","C50821","C50822","C50829","C509","C5091","C50911","C50912",
             "C50919","C5092","C50921","C50922","C50929"],

    '肺癌': ["C33","C34","C340","C3400","C3401","C3402","C341","C3410","C3411","C3412","C342","C343","C3430",
             "C3431","C3432","C348","C3480","C3481","C3482","C349","C3490","C3491","C3492"],

    '胃癌': ["C16","C160","C161","C162","C163","C164","C165","C166","C168","C169"],

    '食道癌': ["C15","C153","C154","C155","C158","C159"],

    '胰臟癌': ["C25","C250","C251","C252","C253","C254","C257","C258","C259"],

    '腸癌': ["C17","C170","C171","C172","C173","C178","C179","C18","C180","C181","C182","C183","C184",
             "C185","C186","C187","C188","C189","C19","C20","C21","C210","C211","C212","C218"],

    '膽囊癌': ["C23","for_tuple_fill"],

    '卵巢癌': ["C56","C561","C562","C563","C569"],

    '糖尿病': ["E08","E080","E0800","E0801","E081","E0810","E0811","E082","E0821","E0822","E0829","E083","E0831",
             "E08311","E08319","E0832","E08321","E08329","E0833","E08331","E08339","E0834","E08341","E08349",
             "E0835","E08351","E08359","E0836","E0839","E084","E0840","E0841","E0842","E0843","E0844","E0849",
             "E085","E0851","E0852","E0859","E086","E0861","E08610","E08618","E0862","E08620","E08621","E08622",
             "E08628","E0863","E08630","E08638","E0864","E08641","E08649","E0865","E0869","E088","E089","E09",
             "E090","E0900","E0901","E091","E0910","E0911","E092","E0921","E0922","E0929","E093","E0931","E09311",
             "E09319","E0932","E09321","E09329","E0933","E09331","E09339","E0934","E09341","E09349","E0935","E09351",
             "E09359","E0936","E0939","E094","E0940","E0941","E0942","E0943","E0944","E0949","E095","E0951","E0952",
             "E0959","E096","E0961","E09610","E09618","E0962","E09620","E09621","E09622","E09628","E0963","E09630",
             "E09638","E0964","E09641","E09649","E0965","E0969","E098","E099","E11","E110","E1100","E1101","E112",
             "E1121","E1122","E1129","E113","E1131","E11311","E11319","E1132","E11321","E11329","E1133","E11331",
             "E11339","E1134","E11341","E11349","E1135","E11351","E11359","E1136","E1139","E114","E1140","E1141",
             "E1142","E1143","E1144","E1149","E115","E1151","E1152","E1159","E116","E1161","E11610","E11618","E1162"
             ,"E11620","E11621","E11622","E11628","E1163","E11630","E11638","E1164","E11641","E11649","E1165","E1169",
             "E118","E119","E13","E130","E1300","E1301","E131","E1310","E1311","E132","E1321","E1322","E1329","E133",
             "E1331","E13311","E13319","E1332","E13321","E13329","E1333","E13331","E13339","E1334","E13341","E13349",
             "E1335","E13351","E13359","E1336","E1339","E134","E1340","E1341","E1342","E1343","E1344","E1349","E135",
             "E1351","E1352","E1359","E136","E1361","E13610","E13618","E1362","E13620","E13621","E13622","E13628","E1363",
             "E13630","E13638","E1364","E13641","E13649","E1365","E1369","E138","E139"],

    '慢性腎病': ["A1811","A5275","C64","C65","C689","D300","D3A093","D410","D411","D422","D593","E082","E092","E1021",
             "E1022","E1029","E1065","E112","E1165","E132","E260","E261","E268","E269","E7203","E723","E728","E740",
             "E744","E748","E7521","E7522","E7524","E753","E77","I12","I13","I701","I722","I7581","I773","I7773",
             "K767","M103","M1A10X0","M1A10X1","M1A1110","M1A1111","M1A1120","M1A1121","M1A1190","M1A1191","M1A1210",
             "M1A1211","M1A1220","M1A1221","M1A1290","M1A1291","M1A1310","M1A1311","M1A1320","M1A1321","M1A1390",
             "M1A1391","M1A1410","M1A1411","M1A1420","M1A1421","M1A1490","M1A1491","M1A1510","M1A1511","M1A1520",
             "M1A1521","M1A1590","M1A1591","M1A1610","M1A1611","M1A1620","M1A1621","M1A1690","M1A1691","M1A1710",
             "M1A1711","M1A1720","M1A1721","M1A1790","M1A1791","M1A18X0","M1A18X1","M1A19X0","M1A19X1","M3214",
             "M3215","M3504","N00","N008","N009","N01","N02","N03","N04","N05","N06","N07","N08","N131","N132",
             "N133","N135","N139","N14","N150","N158","N159","N16","N17","N18","N19","N200","N25","N261","N269",
             "N271","N279","N289","N29","O104","O121","O2683","Q60","Q6102","Q611","Q612","Q613","Q614","Q615",
             "Q618","Q620","Q621","Q622","Q6231","Q6232","Q6239","Q63","Q851","R944","T560X1A","T560X2A","T560X3A",
             "T560X4A","Z4822","Z524","Z940","Z99"],
    
    '阿茲海默症': ["G30","G300","G01","G08","G09"]

}

#計算Row data
for disease_group, codes in group_disease_mapping.items():
    
    #檢查/創立各病儲存夾
    save_path = forward_path + disease_group
    if not os.path.isdir(save_path):
        os.mkdir(save_path)
    
    #聯集三表間id
    CRLF = pd.read_sql("SELECT id, site FROM CRLF", conn)
    CRLF = CRLF[CRLF['site'].isin(codes)]
    CRLF_id_list = CRLF['id'].tolist()
    
    TOTFAE = pd.read_sql("SELECT d3, d19, d20, d21, d22, d23 FROM TOTFAE", conn)
    TOTFAE = TOTFAE[TOTFAE[['d19', 'd20', 'd21', 'd22', 'd23']].isin(codes).any(axis=1)]
    TOTFAE_id_list = TOTFAE['d3'].tolist()
    
    TOTFBE = pd.read_sql("SELECT d3, d25, d26, d27, d28, d29 FROM TOTFBE", conn)
    TOTFBE = TOTFBE[TOTFBE[['d25', 'd26', 'd27', 'd28', 'd29']].isin(codes).any(axis=1)]
    TOTFBE_id_list = TOTFBE['d3'].tolist()
    
    union_set = set(CRLF_id_list).union(TOTFAE_id_list, TOTFBE_id_list)
    union_list = list(union_set)
    
    #以id查詢/儲存資料
    CRLF_save_query = "SELECT * FROM CRLF WHERE id IN ({})".format(','.join('?' for _ in union_list))
    CRLF_save = pd.read_sql(CRLF_save_query, conn, params=union_list)
    CRLF_save = CRLF_save.to_json(save_path+"\\CRLF.json",orient='records',date_format = 'iso', force_ascii=False)
    
    TOTFAE_save_query = "SELECT * FROM TOTFAE WHERE d3 IN ({})".format(','.join('?' for _ in union_list))
    TOTFAE_save = pd.read_sql(TOTFAE_save_query, conn, params=union_list)
    TOTFAE_save = TOTFAE_save.to_json(save_path+"\\TOTFAE.json",orient='records',date_format = 'iso', force_ascii=False)
    
    TOTFBE_save_query = "SELECT * FROM TOTFBE WHERE d3 IN ({})".format(','.join('?' for _ in union_list))
    TOTFBE_save = pd.read_sql(TOTFBE_save_query, conn, params=union_list)
    TOTFBE_save = TOTFBE_save.to_json(save_path+"\\TOTFBE.json",orient='records',date_format = 'iso', force_ascii=False)
    
    TOTFAO1_d_save_query = "SELECT * FROM TOTFAO1 WHERE d3 IN ({}) AND LENGTH(p4) = 10".format(','.join('?' for _ in union_list))
    TOTFAO1_d_save = pd.read_sql(TOTFAO1_d_save_query, conn, params=union_list)
    TOTFAO1_d_save = TOTFAO1_d_save.to_json(save_path+"\\TOTFAO1_d.json",orient='records',date_format = 'iso', force_ascii=False)
    
    TOTFAO1_sc_save_query = "SELECT * FROM TOTFAO1 WHERE d3 IN ({}) AND LENGTH(p4) BETWEEN 6 AND 7".format(','.join('?' for _ in union_list))
    TOTFAO1_sc_save = pd.read_sql(TOTFAO1_sc_save_query, conn, params=union_list)
    TOTFAO1_sc_save = TOTFAO1_sc_save.to_json(save_path+"\\TOTFAO1_sc.json",orient='records',date_format = 'iso', force_ascii=False)
    
    TOTFAO1_o_save_query = "SELECT * FROM TOTFAO1 WHERE d3 IN ({}) AND LENGTH(p4) NOT IN (6, 7, 10)".format(','.join('?' for _ in union_list))
    TOTFAO1_o_save = pd.read_sql(TOTFAO1_o_save_query, conn, params=union_list)
    TOTFAO1_o_save = TOTFAO1_o_save.to_json(save_path+"\\TOTFAO1_o.json",orient='records',date_format = 'iso', force_ascii=False)
    
    TOTFAO2_save_query = "SELECT * FROM TOTFAO2 WHERE d3 IN ({})".format(','.join('?' for _ in union_list))
    TOTFAO2_save = pd.read_sql(TOTFAO2_save_query, conn, params=union_list)
    TOTFAO2_save = TOTFAO2_save.to_json(save_path+"\\TOTFAO2.json",orient='records',date_format = 'iso', force_ascii=False)
    
    TOTFBO1_d_save_query = "SELECT * FROM TOTFBO1 WHERE d3 IN ({}) AND LENGTH(p3) = 10".format(','.join('?' for _ in union_list))
    TOTFBO1_d_save = pd.read_sql(TOTFBO1_d_save_query, conn, params=union_list)
    TOTFBO1_d_save = TOTFBO1_d_save.to_json(save_path+"\\TOTFBO1_d.json",orient='records',date_format = 'iso', force_ascii=False)
    
    TOTFBO1_sc_save_query = "SELECT * FROM TOTFBO1 WHERE d3 IN ({}) AND LENGTH(p3) BETWEEN 6 AND 7".format(','.join('?' for _ in union_list))
    TOTFBO1_sc_save = pd.read_sql(TOTFBO1_sc_save_query, conn, params=union_list)
    TOTFBO1_sc_save = TOTFBO1_sc_save.to_json(save_path+"\\TOTFBO1_sc.json",orient='records',date_format = 'iso', force_ascii=False)
    
    TOTFBO1_o_save_query = "SELECT * FROM TOTFBO1 WHERE d3 IN ({}) AND LENGTH(p3) NOT IN (6, 7, 10)".format(','.join('?' for _ in union_list))
    TOTFBO1_o_save = pd.read_sql(TOTFBO1_o_save_query, conn, params=union_list)
    TOTFBO1_o_save = TOTFBO1_o_save.to_json(save_path+"\\TOTFBO1_o.json",orient='records',date_format = 'iso', force_ascii=False)
    
    TOTFBO2_save_query = "SELECT * FROM TOTFBO2 WHERE d3 IN ({})".format(','.join('?' for _ in union_list))
    TOTFBO2_save = pd.read_sql(TOTFBO2_save_query, conn, params=union_list)
    TOTFBO2_save = TOTFBO2_save.to_json(save_path+"\\TOTFBO2.json",orient='records',date_format = 'iso', force_ascii=False)
    
    LABM1_save_query = "SELECT * FROM LABM1 WHERE h9 IN ({})".format(','.join('?' for _ in union_list))
    LABM1_save = pd.read_sql(LABM1_save_query, conn, params=union_list)
    LABM1_save = LABM1_save.to_json(save_path+"\\LABM1.json",orient='records',date_format = 'iso', force_ascii=False)
    
    LABM2_save_query = "SELECT * FROM LABM2 WHERE h9 IN ({})".format(','.join('?' for _ in union_list))
    LABM2_save = pd.read_sql(LABM2_save_query, conn, params=union_list)
    LABM2_save = LABM2_save.to_json(save_path+"\\LABM2.json",orient='records',date_format = 'iso', force_ascii=False)
    
    CASE_save_query = "SELECT * FROM [CASE] WHERE id IN ({})".format(','.join('?' for _ in union_list))
    CASE_save = pd.read_sql(CASE_save_query, conn, params=union_list)
    CASE_save = CASE_save.to_json(save_path+"\\CASE.json",orient='records',date_format = 'iso', force_ascii=False)
    
    DEATH_save_query = "SELECT * FROM DEATH WHERE id IN ({})".format(','.join('?' for _ in union_list))
    DEATH_save = pd.read_sql(DEATH_save_query, conn, params=union_list)
    DEATH_save = DEATH_save.to_json(save_path+"\\DEATH.json",orient='records',date_format = 'iso', force_ascii=False)

################################
#基本統計量
################################

#取得當前全部疾病夾路徑
directory_path = r'.\tmp_json_central'
folder_paths = [] 
for root, dirs, files in os.walk(directory_path):
    for dir_name in dirs:
        folder_path = os.path.join(root, dir_name)
        folder_paths.append(folder_path)

#主def
def process_table(disease_name, table_name, json_path, id_column, date_column):
    try:
        df = pd.read_json(json_path, orient='records')

        # 計算每個 'id' 的資料筆數
        id_counts = df[id_column].value_counts()
        
        # 計算每個 'id' 分男女的資料筆數
        if 'gender' in df.columns:
            gender_1_count = len(df[df['gender'] == 1][id_column].unique())
            gender_2_count = len(df[df['gender'] == 2][id_column].unique())
        else:
            gender_1_count = "-"
            gender_2_count = "-"

        # 計算統計數據
        result_df = pd.DataFrame({
            'disease_name': [disease_name],
            'table_name': [table_name],
            'N': [len(id_counts)],
            'N_male': [gender_1_count],
            'N_female': [gender_2_count],
            'Mean': [id_counts.mean()],
            'Min': [id_counts.min()],
            'Q1': [id_counts.quantile(0.25)],
            'Q2': [id_counts.quantile(0.5)],
            'Q3': [id_counts.quantile(0.75)],
            'Max': [id_counts.max()],
            'start_date': [df[date_column].min()],
            'end_date': [df[date_column].max()]
        })

    except:
        result_df = pd.DataFrame({
            'disease_name': [disease_name],
            'table_name': [table_name],
            'N': ["-"],
            'N_male': ["-"],
            'N_female': ["-"],
            'Mean': ["-"],
            'Min': ["-"],
            'Q1': ["-"],
            'Q2': ["-"],
            'Q3': ["-"],
            'Max': ["-"],
            'start_date': ["-"],
            'end_date': ["-"]
        })

    return result_df

        
## 各病開始呼叫主def計算
# 創建表格模板
column_names = ['disease_name', 'table_name', 'N', 'N_male', 'N_female', 'Mean', 'Min', 'Q1', 'Q2', 'Q3', 'Max', 'start_date', 'end_date']
final_summary_df = pd.DataFrame(columns=column_names)

#取code與date的對應欄位
tables = {
    'CRLF': ['CRLF.json', 'id', 'didiag'],
    'TOTFAE': ['TOTFAE.json', 'd3', 'd9'],
    'TOTFBE': ['TOTFBE.json', 'd3', 'd10'],
    'TOTFAO1_d':['TOTFAO1_d.json','d3','d9'],
    'TOTFAO1_sc':['TOTFAO1_sc.json','d3','d9'],
    'TOTFAO1_o':['TOTFAO1_o.json','d3','d9'],
    'TOTFAO2':['TOTFAO2.json','d3','d9'],
    'TOTFBO1_d':['TOTFBO1_d.json','d3','d9'],
    'TOTFBO1_sc':['TOTFBO1_sc.json','d3','d9'],
    'TOTFBO1_o':['TOTFBO1_o.json','d3','d9'],
    'TOTFBO2':['TOTFBO2.json','d3','d9'],
    'LABM1':['LABM1.json','h9','h13'],
    'LABM2':['LABM2.json','h9','h13'],
    'CASE':['CASE.json','id','m3'],
    'DEATH':['DEATH.json','id','d4'],
}

##疊/儲存表格
for path in folder_paths:
    
    #從資料夾名稱取出disease_name
    disease_name = os.path.basename(path)
    
    for table_name, (json_file, id_column, date_column) in tables.items():
        json_path = os.path.join(path, json_file)
        summary_df = process_table(disease_name, table_name, json_path, id_column, date_column)
        final_summary_df = pd.concat([final_summary_df, summary_df], ignore_index=True)

final_summary_df = final_summary_df.round(2)
final_summary_df_save = final_summary_df.to_json("./tmp_json_central/final_summary_df.json",orient='records',date_format = 'iso', force_ascii=False)

################################
#醫令細項統計量
################################

##找出母資料夾底下所有AO1、BO1
def search_files(directory, prefix_list):
    
    matching_files_AE = []
    matching_files_BE = []
    
    for root, dirs, files in os.walk(directory):
        for file in files:
            for prefix in prefix_list:
                if file.startswith(prefix):
                    if prefix == 'TOTFAO1':
                        matching_files_AE.append(os.path.join(root, file))
                    elif prefix == 'TOTFBO1':
                        matching_files_BE.append(os.path.join(root, file))
    
    return matching_files_AE, matching_files_BE

# 指定母資料夾路徑
directory = "./tmp_json_central"

# 指定檔案名稱的前綴
prefix_list = ['TOTFAO1', 'TOTFBO1']

# 搜尋符合條件的檔案並排除二跑既存_freq檔
matching_files_AE, matching_files_BE = search_files(directory, prefix_list)
matching_files_AE = [f for f in matching_files_AE if not re.search(r'_freq\.json$', f)]
matching_files_BE = [f for f in matching_files_BE if not re.search(r'_freq\.json$', f)]

##主def_AE
def sql_query_AE(json_path):
    
    df = pd.read_json(json_path, orient ='records')
    
    #宣告 pysqldf 函數通道 且 轉為local變數配合區域變數df
    pysqldf = partial(sqldf, env=locals())
        
    # SQL查詢語法
    query = """
        WITH patient_usage AS (
            SELECT 
                p4, 
                d3, 
                COUNT(*) as usage_count
            FROM 
                df
            GROUP BY 
                p4, d3
        ),
        total_usage AS (
            SELECT 
                p4,
                COUNT(*) as total_count
            FROM 
                df
            GROUP BY 
                p4
        ),
        quartile_stats AS (
            SELECT
                p4,
                AVG(usage_count) FILTER (WHERE quartile = 1) AS Q1,
                AVG(usage_count) FILTER (WHERE quartile = 2) AS Q2,
                AVG(usage_count) FILTER (WHERE quartile = 3) AS Q3,
                AVG(usage_count) FILTER (WHERE quartile = 4) AS Q4
            FROM (
                SELECT
                    p4,
                    usage_count,
                    NTILE(4) OVER (PARTITION BY p4 ORDER BY usage_count) AS quartile
                FROM 
                    patient_usage
            )
            GROUP BY p4
        ),
        order_stats AS (
            SELECT 
                pu.p4, 
                COUNT(*) as unique_users, 
                AVG(pu.usage_count) as avg_usage_per_user,
                MAX(pu.usage_count) as max_usage_per_user, 
                MIN(pu.usage_count) as min_usage_per_user,
                tu.total_count
            FROM 
                patient_usage pu
            JOIN 
                total_usage tu ON pu.p4 = tu.p4
            GROUP BY 
                pu.p4, tu.total_count
        ),
        time_stats AS (
            SELECT 
                p4, 
                MIN(t3) as first_usage_date, 
                MAX(t3) as last_usage_date
            FROM 
                df
            GROUP BY 
                p4
        )
        SELECT 
            o.p4, 
            o.unique_users, 
            o.avg_usage_per_user, 
            o.max_usage_per_user, 
            o.min_usage_per_user,
            o.total_count,
            q.Q1,
            q.Q2,
            q.Q3,
            q.Q4,
            t.first_usage_date, 
            t.last_usage_date
        FROM 
            order_stats o
        JOIN 
            quartile_stats q ON o.p4 = q.p4
        JOIN 
            time_stats t ON o.p4 = t.p4;
    """
    # 查詢
    try:
        result = pysqldf(query)
        result = result.round(2)

        # 另存新檔 base_name:路徑串 ; extension:副檔名
        base_name, extension = os.path.splitext(json_path)
        new_path = base_name + '_freq' + extension
        df_save = result.to_json(new_path,orient='records',date_format = 'iso', force_ascii=False)

        return('Success: ',new_path)
    
    except:
        result = pd.DataFrame()
        
        # 另存新檔 base_name:路徑串 ; extension:副檔名
        base_name, extension = os.path.splitext(json_path)
        new_path = base_name + '_freq' + extension
        df_save = result.to_json(new_path,orient='records',date_format = 'iso', force_ascii=False)

        return('Fail: ',new_path)
        
##主def_BE
def sql_query_BE(json_path):
    
    df = pd.read_json(json_path, orient ='records')
    
    #宣告 pysqldf 函數通道 且 轉為local變數配合區域變數df
    pysqldf = partial(sqldf, env=locals())
    
    # SQL查詢語法
    query = """
        WITH patient_usage AS (
            SELECT 
                p3, 
                d3, 
                COUNT(*) as usage_count
            FROM 
                df
            GROUP BY 
                p3, d3
        ),
        total_usage AS (
            SELECT 
                p3,
                COUNT(*) as total_count
            FROM 
                df
            GROUP BY 
                p3
        ),
        quartile_stats AS (
            SELECT
                p3,
                AVG(usage_count) FILTER (WHERE quartile = 1) AS Q1,
                AVG(usage_count) FILTER (WHERE quartile = 2) AS Q2,
                AVG(usage_count) FILTER (WHERE quartile = 3) AS Q3,
                AVG(usage_count) FILTER (WHERE quartile = 4) AS Q4
            FROM (
                SELECT
                    p3,
                    usage_count,
                    NTILE(4) OVER (PARTITION BY p3 ORDER BY usage_count) AS quartile
                FROM 
                    patient_usage
            )
            GROUP BY p3
        ),
        order_stats AS (
            SELECT 
                pu.p3, 
                COUNT(*) as unique_users, 
                AVG(pu.usage_count) as avg_usage_per_user,
                MAX(pu.usage_count) as max_usage_per_user, 
                MIN(pu.usage_count) as min_usage_per_user,
                tu.total_count
            FROM 
                patient_usage pu
            JOIN 
                total_usage tu ON pu.p3 = tu.p3
            GROUP BY 
                pu.p3, tu.total_count
        ),
        time_stats AS (
            SELECT 
                p3, 
                MIN(t3) as first_usage_date, 
                MAX(t3) as last_usage_date
            FROM 
                df
            GROUP BY 
                p3
        )
        SELECT 
            o.p3, 
            o.unique_users, 
            o.avg_usage_per_user, 
            o.max_usage_per_user, 
            o.min_usage_per_user,
            o.total_count,
            q.Q1,
            q.Q2,
            q.Q3,
            q.Q4,
            t.first_usage_date, 
            t.last_usage_date
        FROM 
            order_stats o
        JOIN 
            quartile_stats q ON o.p3 = q.p3
        JOIN 
            time_stats t ON o.p3 = t.p3;
    """ 
    # 查詢
    try:     
        result = pysqldf(query)
        result = result.round(2)

        # 另存新檔 base_name:路徑串 ; extension:副檔名
        base_name, extension = os.path.splitext(json_path)
        new_path = base_name + '_freq' + extension
        df_save = result.to_json(new_path,orient='records',date_format = 'iso', force_ascii=False)

        return(new_path)
    
    except:
        result = pd.DataFrame()

        # 另存新檔 base_name:路徑串 ; extension:副檔名
        base_name, extension = os.path.splitext(json_path)
        new_path = base_name + '_freq' + extension
        df_save = result.to_json(new_path,orient='records',date_format = 'iso', force_ascii=False)

        return('Fail: ',new_path)
        
# AE迴圈
for json_path in matching_files_AE:
    sql_query_AE(json_path)

# BE迴圈
for json_path in matching_files_BE:
    sql_query_BE(json_path)

################################
#刪除row data後壓縮供下載
################################

# 定義刪除夾
delete_path = './tmp_json_central'

# 遍歷
for root, dirs, files in os.walk(delete_path, topdown=False):
    for name in files:
        # 建構完整的檔案路徑
        file_path = os.path.join(root, name)
        # 檢查檔案名是否不以'_freq'结尾
        if not name.endswith(('_freq.json','final_summary_df.json')):
            # 删除不以'_freq'结尾的文件及summary檔
            os.remove(file_path)

# 定義zip壓縮檔案的名稱與被壓縮夾
zip_name = os.path.join('./tmp_json_central_download/', 'central_result')
zip_path ='./tmp_json_central/'

# 創建壓縮檔案
shutil.make_archive(zip_name, 'zip', zip_path)

'E:\\NBCT_group_count\\tmp_json_central_download\\central_result.zip'